In [2]:
import torch
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from torch import nn

from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!pip install emoji

In [4]:

import re
import string
import emoji
import nltk
import spacy
import scipy.sparse
from tqdm import tqdm
from nltk.corpus import stopwords
from gensim.models import Word2Vec

In [5]:
Data =pd.read_csv('/content/drive/MyDrive/Colab Notebooks/projects/disaster/train.csv')
result =pd.read_csv('/content/drive/MyDrive/Colab Notebooks/projects/disaster/test.csv')

Data[Data['location']=='WorldWide'].sample(frac=.50)

,id,keyword,location,text,target
7510,10743,wreckage,WorldWide,#Australia #News ; RT janeenorman: 'High proba...,1
3813,5417,first%20responders,WorldWide,Loved the way this book was written to include...,0


In [6]:
data=(Data.groupby('keyword').sum()==0)

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 221 entries, ablaze to wrecked
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   id        221 non-null    bool 
 1   location  221 non-null    bool 
 2   text      221 non-null    bool 
 3   target    221 non-null    bool 
dtypes: bool(4)
memory usage: 2.6+ KB


In [8]:

def describe(df):
  desc=pd.DataFrame()
  desc['dtype']=df.dtypes
  desc['null']=df.isnull().sum()
  desc['%null'] = desc['null'] / len(df) * 100
  desc['unique']=df.nunique()
  #desc['%unique'] = desc['nunique'] /len(df)*100
  desc = pd.concat([desc,df.describe(include='all').round(2).T],axis=1)
  display(desc)
describe(Data)


,dtype,null,%null,unique,count,unique,top,freq,mean,std,min,25%,50%,75%,max
id,int64,0,0.000000,7613,7613.0,NaN,NaN,NaN,5441.93,3137.12,1.0,2734.0,5408.0,8146.0,10873.0
keyword,object,61,0.801261,221,7552,221,fatalities,45,NaN,NaN,NaN,NaN,NaN,NaN,NaN
location,object,2533,33.272035,3341,5080,3341,USA,104,NaN,NaN,NaN,NaN,NaN,NaN,NaN
text,object,0,0.000000,7503,7613,7503,11-Year-Old Boy Charged With Manslaughter of T...,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN
target,int64,0,0.000000,2,7613.0,NaN,NaN,NaN,0.43,0.5,0.0,0.0,0.0,1.0,1.0


In [9]:
df=Data.drop(columns=['id'])

In [10]:
df['text'] = df['text'].str.lower()

In [11]:
def remove_html_tags(text):
    clean_text = re.sub('<.*?>', '', text)
    return clean_text

df['text'] = df['text'].apply(remove_html_tags)



In [12]:
def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    clean_text = re.sub(url_pattern, '', text)
    return clean_text

df['text'] = df['text'].apply(remove_urls)



In [13]:
def remove_punctuation(text):
    punctuation = string.punctuation
    clean_text = text.translate(str.maketrans('', '', punctuation))
    return clean_text

df['text'] = df['text'].apply(remove_punctuation)



In [14]:


chat_words_mapping = {
    "lol": "laughing out loud",
    "brb": "be right back",
    "btw": "by the way",
    "afk": "away from keyboard",
    "rofl": "rolling on the floor laughing",
    "ttyl": "talk to you later",
    "np": "no problem",
    "thx": "thanks",
    "omg": "oh my god",
    "idk": "I don't know",
    "np": "no problem",
    "gg": "good game",
    "g2g": "got to go",
    "b4": "before",
    "cu": "see you",
    "yw": "you're welcome",
    "wtf": "what the f*ck",
    "imho": "in my humble opinion",
    "jk": "just kidding",
    "gf": "girlfriend",
    "bf": "boyfriend",
    "u": "you",
    "r": "are",
    "2": "to",
    "4": "for",
    "b": "be",
    "c": "see",
    "y": "why",
    "tho": "though",
    "smh": "shaking my head",
    "lolz": "laughing out loud",
    "h8": "hate",
    "luv": "love",
    "pls": "please",
    "sry": "sorry",
    "tbh": "to be honest",
    "omw": "on my way",
    "omw2syg": "on my way to see your girlfriend",
}

def expand_chat_words(text):
    words = text.split()
    expanded_words = [chat_words_mapping.get(word.lower(), word) for word in words]
    return ' '.join(expanded_words)

df['text'] = df['text'].apply(expand_chat_words)



In [15]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [16]:


def remove_stop_words(text):
	tokens = nltk.word_tokenize(text)
	stop_words = set(stopwords.words('english'))
	filtered_tokens = [token for token in tokens if token not in stop_words]
	preprocessed_text = ' '.join(filtered_tokens)
	return preprocessed_text

df['text'] = df['text'].apply(remove_stop_words)



In [17]:
def replace_emojis_with_meanings(text):
    def replace(match):
        emoji_char = match.group()
        emoji_meaning = emoji.demojize(emoji_char)
        return emoji_meaning

    emoji_pattern = re.compile("["
                            u"\U0001F600-\U0001F64F"
                            u"\U0001F300-\U0001F5FF"
                            u"\U0001F680-\U0001F6FF"
                            u"\U0001F1E0-\U0001F1FF"
                            u"\U00002500-\U00002BEF"
                            u"\U00002702-\U000027B0"
                            u"\U00002702-\U000027B0"
                            u"\U000024C2-\U0001F251"
                            u"\U0001f926-\U0001f937"
                            u"\U00010000-\U0010ffff"
                            u"\u2640-\u2642"
                            u"\u2600-\u2B55"
                            u"\u200d"
                            u"\u23cf"
                            u"\u23e9"
                            u"\u231a"
                            u"\ufe0f"
                            u"\u3030"
                            "]+", flags=re.UNICODE)
    text_with_meanings = emoji_pattern.sub(replace, text)
    return text_with_meanings

df['text'] = df['text'].apply(replace_emojis_with_meanings)



In [18]:
display(df)

,keyword,location,text,target
0,NaN,NaN,deeds reason earthquake may allah forgive us,1
1,NaN,NaN,forest fire near la ronge sask canada,1
2,NaN,NaN,residents asked shelter place notified officer...,1
3,NaN,NaN,13000 people receive wildfires evacuation orde...,1
4,NaN,NaN,got sent photo ruby alaska smoke wildfires pou...,1
...,...,...,...,...
7608,NaN,NaN,two giant cranes holding bridge collapse nearb...,1
7609,NaN,NaN,ariaahrary thetawniest control wild fires cali...,1
7610,NaN,NaN,m194 0104 utc5km volcano hawaii,1
7611,NaN,NaN,police investigating ebike collided car little...,1


In [30]:
import transformers

from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_name = 'google-bert/bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_name)


# checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
# model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

# for i in df['text'] :
#  inputs = tokenizer(i, padding=True, truncation=True, return_tensors="pt")
# #  outputs = model(**inputs)


In [ ]:
#

In [20]:


def tokenize_function(example):
    return tokenizer(example,  padding=True,truncation=True,return_tensors="pt")


tokenized_datasets = df['text'].apply(tokenize_function)
tokenized_datasets[1]

{'input_ids': tensor([[  101,  3224,  2543,  2379,  2474,  6902,  3351, 21871,  2243,  2710,
           102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [21]:
from transformers import DataCollatorWithPadding
from torch.utils.data import DataLoader


data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

data_collator=DataLoader(tokenized_datasets,batch_size=16,shuffle=True,collate_fn=data_collator)

In [ ]:
!pip install transformers[torch]




In [ ]:
tokenized_datasets.shape[:]

In [22]:
!pip install torchmetrics

In [23]:
import torchmetrics

def compute_metrics(eval_pred):
    metric = torchmetrics.F1Score(task='multiclass', num_classes=2)
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {'F1 Score': metric(torch.tensor(predictions), torch.tensor(labels)).item()}

In [25]:
! pip install -U accelerate
! pip install -U transformers

In [33]:


from transformers import TrainingArguments

training_args = TrainingArguments( output_dir='classifier_model',
    eval_strategy="epoch",
    save_strategy="epoch",

    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    load_best_model_at_end=True,
    num_train_epochs=3,
    weight_decay=0.01,)
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets[:1522],
    eval_dataset=tokenized_datasets[1522:],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)



In [34]:
trainer.train()


ValueError: Unable to create tensor, you should probably activate truncation and/or padding with 'padding=True' 'truncation=True' to have batched tensors with the same length. Perhaps your features (`input_ids` in this case) have excessive nesting (inputs type `list` where type `int` is expected).

In [ ]:
class TextLengthExtractor(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        return pd.DataFrame(X['text'].apply(len))

class NumHashtagsExtractor(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        return pd.DataFrame(X['text'].apply(lambda x: len([word for word in x.split() if word.startswith('#')])))

class NumMentionsExtractor(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        return pd.DataFrame(X['text'].apply(lambda x: len([word for word in x.split() if word.startswith('@')])))

5.2. Train Test Split

X = df[['text']]
y = df['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

5.3. Creating Pipeline for model

pipeline = Pipeline([
    ('features', FeatureUnion([
        ('text', Pipeline([
            ('selector', FunctionTransformer(lambda x: x['text'], validate=False)),
            ('tfidf', TfidfVectorizer(max_features=10000))
        ])),
        ('text_length', Pipeline([
            ('selector', FunctionTransformer(lambda x: x, validate=False)),
            ('extract', TextLengthExtractor())
        ])),
        ('num_hashtags', Pipeline([
            ('selector', FunctionTransformer(lambda x: x, validate=False)),
            ('extract', NumHashtagsExtractor())
        ])),
        ('num_mentions', Pipeline([
            ('selector', FunctionTransformer(lambda x: x, validate=False)),
            ('extract', NumMentionsExtractor())
        ]))
    ])),
    ('clf', GradientBoostingClassifier(n_estimators=100, learning_rate=0.1))
])

param_grid = {
    'features__text__tfidf__max_features': [5000, 10000],
    'clf__n_estimators': [100, 200],
    'clf__learning_rate': [0.1, 0.01]
}

grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='f1', n_jobs=1)
grid_search.fit(X_train, y_train)

from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,confusion_matrix,classification_report
best_model = grid_search.best_estimator_
y_pred_best = best_model.predict(X_test)
print(classification_report(y_test, y_pred_best))




In [ ]:
df.head()

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn import feature_extraction, linear_model, model_selection, preprocessing

count_vectorizer = feature_extraction.text.CountVectorizer()
train_vectors = count_vectorizer.fit_transform(Data["text"])

## note that we're NOT using .fit_transform() here. Using just .transform() makes sure
# that the tokens in the train vectors are the only ones mapped to the test vectors -
# i.e. that the train and test vectors use the same set of tokens.
test_vectors = count_vectorizer.transform(Data["text"])

from sklearn.ensemble import RandomForestClassifier
model=RandomForestClassifier(n_estimators=40)

scores = model_selection.cross_val_score(model, train_vectors, Data["target"], cv=3, scoring="f1")
scores
# clf.fit(train_vectors, Data["target"])

In [ ]:
display(pd.DataFrame(test_vectors))